# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [10]:
# import libraries
import numpy as np
import pandas as pd
import re
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split,  GridSearchCV 
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

from sqlalchemy import create_engine
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Romullo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Romullo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Romullo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table("DisasterResponse", con=engine)

categories = df.columns[4:]

X = df[['message']].values[:, 0]
y = df[categories].values

In [28]:
type(y)

numpy.ndarray

In [5]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [14]:
def tokenize(text):
       
       
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    # tokenize text
    tokens = word_tokenize(text)
    
    # remove stopwords
    #tokens = [t for t in tokens if t not in stopwords.words('english')]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [15]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
# Spiliting data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [18]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002A948936160>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
 # predict on test data
y_pred = pipeline.predict(X_test)

# display results
display_results(y_test[0], y_pred[0])
#display_results(y_test, y_pred)

Labels: [0 1]
Confusion Matrix:
 [[31  0]
 [ 1  4]]
Accuracy: 0.9722222222222222


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
# Show parameters for the pipline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000002A948936160>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000002A948936160>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [21]:
parameters = {
    'clf__estimator__n_estimators': [10, 20]
}

cv = GridSearchCV(pipeline, param_grid = parameters)
cv

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002A948936160>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
# Fit model
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002A948936160>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20]})

In [23]:
# Predicting model 
y_pred = cv.predict(X_test)
#multioutput_classification_report(y_test, y_pred)

In [24]:
columns = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 
       'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 
       'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 
       'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

In [25]:
for i,col in enumerate(columns):
    print(col)
    accuracy = accuracy_score(y_test[i], y_pred[i])
    precision = precision_score(y_test[i], y_pred[i])
    recall = recall_score(y_test[i], y_pred[i])
    print("\tAccuracy: %.2f\tPrecision: %.2f\t Recall: %.2f\n" % (accuracy, precision, recall))

related
	Accuracy: 0.97	Precision: 1.00	 Recall: 0.80

request
	Accuracy: 0.94	Precision: 1.00	 Recall: 0.33

offer
	Accuracy: 0.97	Precision: 0.00	 Recall: 0.00

aid_related
	Accuracy: 1.00	Precision: 1.00	 Recall: 1.00

medical_help
	Accuracy: 0.97	Precision: 1.00	 Recall: 0.67

medical_products
	Accuracy: 0.92	Precision: 1.00	 Recall: 0.67

search_and_rescue
	Accuracy: 0.94	Precision: 1.00	 Recall: 0.71

security
	Accuracy: 0.94	Precision: 1.00	 Recall: 0.60

military
	Accuracy: 0.97	Precision: 0.00	 Recall: 0.00

child_alone
	Accuracy: 0.94	Precision: 0.00	 Recall: 0.00

water
	Accuracy: 1.00	Precision: 0.00	 Recall: 0.00

food
	Accuracy: 0.86	Precision: 1.00	 Recall: 0.17

shelter
	Accuracy: 0.97	Precision: 1.00	 Recall: 0.67

clothing
	Accuracy: 0.97	Precision: 0.00	 Recall: 0.00

money
	Accuracy: 0.92	Precision: 1.00	 Recall: 0.40

missing_people
	Accuracy: 0.97	Precision: 1.00	 Recall: 0.75

refugees
	Accuracy: 0.94	Precision: 0.33	 Recall: 1.00

death
	Accuracy: 1.00	Precision

C:\Users\Romullo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Romullo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


	Accuracy: 1.00	Precision: 1.00	 Recall: 1.00

electricity
	Accuracy: 0.92	Precision: 1.00	 Recall: 0.40

tools
	Accuracy: 0.97	Precision: 1.00	 Recall: 0.67

hospitals
	Accuracy: 0.97	Precision: 0.50	 Recall: 1.00

shops
	Accuracy: 0.97	Precision: 0.00	 Recall: 0.00

aid_centers
	Accuracy: 0.94	Precision: 0.75	 Recall: 0.75

other_infrastructure
	Accuracy: 0.86	Precision: 0.00	 Recall: 0.00

weather_related
	Accuracy: 0.89	Precision: 1.00	 Recall: 0.20

floods
	Accuracy: 0.94	Precision: 1.00	 Recall: 0.33

storm
	Accuracy: 0.92	Precision: 1.00	 Recall: 0.25

fire
	Accuracy: 1.00	Precision: 1.00	 Recall: 1.00

earthquake
	Accuracy: 0.97	Precision: 0.00	 Recall: 0.00

cold
	Accuracy: 0.94	Precision: 1.00	 Recall: 0.50

other_weather
	Accuracy: 0.94	Precision: 0.33	 Recall: 1.00

direct_report
	Accuracy: 0.92	Precision: 0.00	 Recall: 0.00



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [26]:
file_name = 'classifier.pkl'
with open (file_name, 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.